In [21]:
!nvidia-smi

Thu Jan 14 18:06:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    28W /  70W |   7677MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from transformers import XLMRobertaTokenizer, DistilBertForSequenceClassification, \
                         DistilBertTokenizerFast, XLMRobertaForSequenceClassification

## XLM Roberta Model

In [24]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=1000,
)


In [58]:
xlmroberta = SentimixTransformer(
                                 XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=3),
                                 XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")
                                 )

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [59]:
train_data, test_data, labels = xlmroberta.preprocess_data("../data/processed_train.csv", "clean_text", "labels")

In [61]:
xlmroberta.train(training_args, train_data)

Step,Training Loss


In [62]:
xlmroberta.evaluate(test_data)

{'epoch': 1.0,
 'eval_accuracy': 0.5880306193458594,
 'eval_f1': 0.5693027149916818,
 'eval_loss': 0.868428647518158,
 'eval_precision': 0.5823817952393507,
 'eval_recall': 0.5880306193458594,
 'eval_runtime': 6.1171,
 'eval_samples_per_second': 234.914}

In [31]:
xlmroberta.save_model("../weights/xlm_roberta/")

In [63]:
xlmroberta.build_pipeline()

In [64]:

testset = pd.read_csv('../data/processed_val.csv')
# testset.shape

testset['xlmroberta_output'] = testset['clean_text'].apply(lambda z: xlmroberta.inference_pipeline(z)[0])
testset['xlmroberta_prediction'] = testset['xlmroberta_output'].apply(lambda z: labels[xlmroberta.model.config.label2id[z['label']]])
testset['xlmroberta_confidence'] = testset['xlmroberta_output'].apply(lambda z: z['score'])
testset.to_csv("../data/outputs/testset_xlmroberta_output.csv", index=False)

testset.head()


,id,sentiment,text,language_labels,clean_text,labels,xlmroberta_output,xlmroberta_prediction,xlmroberta_confidence
0,182,negative,@ tamashbeen _ Well chara Chor ke chele this n...,"['O', 'Eng', 'O', 'Eng', 'Eng', 'Hin', 'Hin', ...",mention tamashbeen well chara chor ke chele th...,0,"{'label': 'LABEL_0', 'score': 0.45618289709091...",negative,0.456183
1,701,positive,@ suhailswarraich Journals k bare me . Bt ki i...,"['O', 'Eng', 'Eng', 'Eng', 'Hin', 'Hin', 'O', ...",mention suhailswarraich journals k bare me bt ...,2,"{'label': 'LABEL_0', 'score': 0.5885549783706665}",negative,0.588555
2,1093,negative,@ SKFforlife Haha Overrated kisko bolte hain c...,"['O', 'Hin', 'Hin', 'Hin', 'Hin', 'Hin', 'Hin'...",mention skfforlife haha overrated kisko bolte ...,0,"{'label': 'LABEL_0', 'score': 0.5542877912521362}",negative,0.554288
3,1106,positive,I wanted to see you in Loksabha @ kanhaiyakuma...,"['Eng', 'Eng', 'Eng', 'Eng', 'Eng', 'Eng', 'Hi...",i wanted to see you in loksabha mention kanhai...,2,"{'label': 'LABEL_2', 'score': 0.7313329577445984}",positive,0.731333
4,1156,negative,@ abpnewshindi ABP news 5 saal se mon nahi bha...,"['O', 'Hin', 'Eng', 'Eng', 'O', 'Hin', 'Hin', ...",mention abpnewshindi abp news 5 saal se mon na...,0,"{'label': 'LABEL_0', 'score': 0.6337724924087524}",negative,0.633772


In [65]:
xlmroberta.inference_pipeline("hello ")

[{'label': 'LABEL_1', 'score': 0.5788123607635498}]

## Distilbert Model

In [ ]:
training_args = TrainingArguments(
    output_dir='./results_1',          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs_1',            # directory for storing logs
    logging_steps=1000,
)


In [ ]:
distilbert = SentimixTransformer(
                                 DistilBertForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=3),
                                 DistilBertTokenizerFast.from_pretrained("xlm-roberta-base")
                                 )

In [ ]:
train_data, test_data, labels = distilbert.preprocess_data("../data/processed_train.csv", "clean_text", "labels")

In [ ]:
distilbert.train(training_args, train_data)

In [ ]:
distilbert.evaluate(test_data)

In [ ]:
distilbert.save_model("../weights/distilbert/")

In [ ]:
distilbert.build_pipeline()

In [ ]:
testset = pd.read_csv('../data/processed_val.csv')
# testset.shape

testset['distilbert_output'] = testset['clean_text'].apply(lambda z: distilbert.inference_pipeline(z)[0])
testset['distilbert_prediction'] = testset['distilbert_output'].apply(lambda z: labels[distilbert.model.config.label2id[z['label']]])
testset['distilbert_confidence'] = testset['distilbert_output'].apply(lambda z: z['score'])
testset.to_csv("../data/outputs/testset_distilbert_output.csv", index=False)

testset.head()

In [ ]:
distilbert.inference_pipeline("hello ")